# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!pip install -U scikit-learn

In [56]:
# import libraries
import re
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV

In [40]:
import nltk
nltk.download(['punkt', 'wordnet','stopwords','omw-1.4','averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\un_po\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\un_po\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\un_po\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\un_po\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\un_po\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table(
    "DisasterResponseTable",
    con=engine
)
#assign data
X = df.message.values
Y = df.iloc[:,4:]

In [5]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26023,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26024,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26025,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26026,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [35]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    lower_text = text.lower().strip()
    no_stop_words = re.sub(r"[^a-zA-Z0-9]", " ", lower_text) 
    words = word_tokenize(no_stop_words)
    words = [w for w in words if w not in stopwords.words("english")]
    stemmed = [PorterStemmer().stem(w) for w in words]
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in stemmed]
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001BBD20F48B0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
y_pred = pipeline.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=Y.columns)
print(classification_report(y_test.values, y_pred.values,target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4944
               request       0.84      0.47      0.60      1170
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.70      0.73      2705
          medical_help       0.72      0.08      0.15       518
      medical_products       0.88      0.09      0.16       331
     search_and_rescue       0.82      0.05      0.10       170
              security       0.00      0.00      0.00       118
              military       0.45      0.04      0.08       200
                 water       0.88      0.36      0.51       407
                  food       0.84      0.63      0.72       700
               shelter       0.83      0.36      0.51       572
              clothing       0.86      0.13      0.22        95
                 money       0.71      0.03      0.06       153
        missing_people       1.00      

C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline

In [17]:
 parameters = {
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__min_samples_split': [3, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

In [18]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END clf__estimator__min_samples_split=3, features__text_pipeline__tfidf__use_idf=True; total time=20.8min
[CV] END clf__estimator__min_samples_split=3, features__text_pipeline__tfidf__use_idf=True; total time=21.0min
[CV] END clf__estimator__min_samples_split=3, features__text_pipeline__tfidf__use_idf=True; total time=21.3min
[CV] END clf__estimator__min_samples_split=3, features__text_pipeline__tfidf__use_idf=True; total time=20.7min
[CV] END clf__estimator__min_samples_split=3, features__text_pipeline__tfidf__use_idf=True; total time=20.8min
[CV] END clf__estimator__min_samples_split=3, features__text_pipeline__tfidf__use_idf=False; total time=21.6min
[CV] END clf__estimator__min_samples_split=3, features__text_pipeline__tfidf__use_idf=False; total time=21.2min
[CV] END clf__estimator__min_samples_split=3, features__text_pipeline__tfidf__use_idf=False; total time=22.2min
[CV] END clf__estimator__min_samples_split=3, fea

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000001BBD20F48B0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [3, 4],
                         'features__text_pipeline__tfidf__use_idf': (True,
                                                                     False)},
             verbose=2)

In [21]:
cv.best_score_

0.262383818559644

In [27]:
cv.best_params_

{'clf__estimator__min_samples_split': 3,
 'features__text_pipeline__tfidf__use_idf': True}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
y_pred = cv.predict(X_test)

In [24]:
y_pred = pd.DataFrame(y_pred, columns=Y.columns)
print(classification_report(y_test.values, y_pred.values,target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4944
               request       0.85      0.50      0.63      1170
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.71      0.73      2705
          medical_help       0.76      0.09      0.17       518
      medical_products       0.81      0.05      0.10       331
     search_and_rescue       0.67      0.06      0.11       170
              security       0.00      0.00      0.00       118
              military       0.50      0.06      0.10       200
                 water       0.90      0.42      0.57       407
                  food       0.85      0.61      0.71       700
               shelter       0.85      0.41      0.55       572
              clothing       0.81      0.14      0.23        95
                 money       0.75      0.04      0.07       153
        missing_people       1.00      

C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
print("The test accuracy is: ")
print(accuracy_score(y_test, y_pred)*100)
print(" ")

The test accuracy is: 
26.63285692331335
 


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [44]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags)>0:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [47]:
pipelineBoost = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [48]:
pipelineBoost.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001BBD20FE280>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [49]:
y_predBoost = pipelineBoost.predict(X_test)
y_predBoost = pd.DataFrame(y_predBoost, columns=Y.columns)
print(classification_report(y_test.values, y_predBoost.values,target_names=Y.columns))

                        precision    recall  f1-score   support

               related       0.79      0.96      0.87      4944
               request       0.77      0.51      0.61      1170
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.63      0.69      2705
          medical_help       0.63      0.26      0.37       518
      medical_products       0.68      0.30      0.42       331
     search_and_rescue       0.62      0.19      0.30       170
              security       0.21      0.05      0.08       118
              military       0.53      0.34      0.41       200
                 water       0.75      0.66      0.70       407
                  food       0.82      0.72      0.77       700
               shelter       0.75      0.56      0.65       572
              clothing       0.66      0.44      0.53        95
                 money       0.58      0.26      0.36       153
        missing_people       0.57      

C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, y_predBoost)*100)
print(" ")


The test accuracy is: 
23.267250653142767
 


The Random Forest model has a higher weigted average precision, while the AdaBoost model has a better F1-Score of 0.63. So we will keep the AdaBoost model.

### 9. Export your model as a pickle file

In [57]:
# Save pickle file in the current working directory
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(pipelineBoost, file)



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.